In [2]:
import numpy as nd
import pandas as pd

#Model Selection and Validation
from sklearn.model_selection import train_test_split

In [3]:
all_tweets = pd.read_csv('./tweets.csv')

In [ ]:
# train_tweets = all_tweets[]

# X_train, X_test, y_train, y_test = train_test_split(, y, test_size = 0.2, random_state = 0, shuffle = True)